In [10]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [11]:
data = pd.read_csv('DummiesV2.csv')
data.head()

,À l'avant du véhicule,À l'arrière du véhicule,Conducteur,Passager,Indemne,Tué,Blessé hospitalisé,Blessé léger,Homme,Aucun équippement,...,3 voies de circulation,4 voies de circulation,5 voies de circulation,6 voies de circulation,7 voies de circulation,8 voies de circulation,9 voies de circulation,10 voies de circulation,11 voies de circulation,12 voies de circulation
0,1,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,1,0,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,1,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,1,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
targets = ['Tué', 'Indemne', 'Blessé léger', 'Blessé hospitalisé']
X = data.drop(targets, axis=1)
X = pd.DataFrame(data=StandardScaler().fit_transform(X), columns=X.columns)
Y = data[targets].values.argmax(axis=1)
X_train, X_test,Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
estimator = LogisticRegression(multi_class='multinomial')
selector = RFE(estimator, n_features_to_select=25, step=1)
selector = selector.fit(X_train, Y_train)
selector.support_ 

C:\Users\Erwan\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Erwan\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

array([False, False,  True,  True,  True, False,  True, False, False,
       False, False, False,  True,  True, False, False, False,  True,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True,  True,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True,  True,  True,
        True, False,  True, False, False,  True, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [14]:
[variable for variable, boolean in zip(X.columns.tolist(), selector.support_.tolist()) if boolean]

['Conducteur',
 'Passager',
 'Homme',
 'Ceinture',
 'Deux-roues non-motorisé',
 'Deux-roues motorisé',
 'Véhicule léger',
 "Pas d'obstacle fixe",
 "Pas d'obstacle mobile",
 'Obstacle : Piéton',
 'Obstacle : Véhicule en mouvement',
 'En tournant',
 'Plein jour',
 'Hors agglomération',
 "Collision par l'arrière (2 véhicules)",
 'Collision par le côté (2 véhicules)',
 'Collision en chaîne (3 véhicules ou plus)',
 'Collisions multiples (3 véhicules ou plus)',
 'Autoroute',
 'Voie communale',
 'Circulation bidirectionnelle',
 'Vitesse limitée à 30km/h',
 'Vitesse limitée à 50km/h',
 '60-74ans',
 '75-120ans']

Voir aussi `sklearn.feature_selection.SelectKBest`